In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import ipywidgets

In [ ]:
from pathlib import Path
import os

In [ ]:
#Put here the folder where files are located
dirpath = '/some_folder_with_a_lot_of_slice_and_view_tiff_files/'

In [ ]:
#paths = sorted(Path(dirpath).iterdir(), key=os.path.getmtime)
paths = sorted(Path(dirpath).iterdir(), key=os.path.basename) #OK
paths

In [ ]:
len(paths)

In [ ]:
dataall=None

In [ ]:
#Loads all images
for i in range(len(paths)):
    filepath = paths[i]
    data = tifffile.imread(filepath)
    if dataall is None:
        dataall = np.zeros((len(paths), data.shape[0], data.shape[1]), dtype = data.dtype)
    
    dataall[i,:,:] = data


In [ ]:
dataall.shape

In [ ]:
dataall.dtype

In [ ]:
dataall.max()

In [ ]:
dataall.min()

In [ ]:
tifffile.imwrite('dataall_uint8.tiff', dataall)

In [ ]:
def show_dataall_by_slice(slice=0):
    fig,ax0 = plt.subplots(1,1)
    ax0.imshow(dataall[slice,:,:])
ipywidgets.interact(show_dataall_by_slice, slice= (0, dataall.shape[0], 1), continuous_update=False)


## Filter dataall using the FFT directional filter

In [ ]:
dataall_filtered = np.zeros_like(dataall).astype(dtype=np.float32)

In [ ]:
def get_DirFFTFilter_From_2D(data2d, freq=5):
    data2d_FFT1D = np.fft.rfft(data2d, axis=1)
    data_fft_filter= np.copy(data2d_FFT1D)
    data_fft_filter[:,:freq] = 0 #high pass filter
    data_filter2 = np.fft.irfft(data_fft_filter, axis=1)
    return data_filter2

In [ ]:
#test
dtest = get_DirFFTFilter_From_2D(dataall[0,:,:], freq=5)
plt.imshow(dtest)

In [ ]:
dtest.dtype

In [ ]:
#Do the filtereing of the whole volume
freq=5
for islice in range(dataall.shape[0]):
    dataall_filtered[islice,:,:] = get_DirFFTFilter_From_2D(dataall[islice,:,:], freq)

In [ ]:
def show_dataall_filtered_by_slice(slice=0):
    fig,ax1 = plt.subplots(1,1,figsize=(5,5))
    ax1.imshow(dataall_filtered[slice,:,:])
ipywidgets.interact(show_dataall_filtered_by_slice, slice= (0, dataall_filtered.shape[0], 1), continuous_update=False)

In [ ]:
dataall_filtered.max()

In [ ]:
dataall_filtered.min()

In [ ]:
#Show a cross section
plt.imshow(dataall_filtered[:,200, :60])

In [ ]:
import napari

In [ ]:
napari.view_image(dataall_filtered[18,:, :])

In [ ]:
napari.view_image(dataall_filtered)

Data is in float format, which takes a lot of space. Convert to uint8

In [ ]:
#Copied from RedLionfish helperfunctions.py
import logging
def convertToFloat32AndNormalise(data, normaliseType=None, bResetZero=True):
    '''
    Converts data to float32 format and optionally resets zero and normalises
    Parameters:
        data
        normalisetype: None for no normalisation, 'max' normalises to max, 'sum' to normalise to sum of whole volume
        bResetZero: flags to whether shift the minimum value to zero
    '''
    if data is None:
        return None
    
    ret= data.astype(np.float32)

    vmin= 0
    if bResetZero:
        vmin = ret.min()
    ret = ret-vmin

    if not normaliseType is None:
        normcorr = 1.0
        if normaliseType == 'max':
            #Normalise to maximum value
            normcorr= ret.max()
        else:
            #normalise to sum
            normcorr = np.sum(ret)
        #Check for zero before division
        if normcorr==0.0:
            logging.info("normcorr = 0. Normalisation will be skipped to prevent division by zero.")
        else:
            ret = ret/normcorr
    return ret
    
def convertToUint8AndFullRange(data):
    if data is None:
        return None
        
    res_256 = convertToFloat32AndNormalise(data, normaliseType='max', bResetZero=True)*256
    res_uint8 = res_256.astype('uint8')
    return res_uint8

In [ ]:
dataall_filtered_uint8 = convertToUint8AndFullRange(dataall_filtered)

In [ ]:
tifffile.imwrite('dataall_filtered_uint8.tiff', dataall_filtered_uint8)